In [1]:
import sys

import numpy as np

sys.path.append("/Users/david/Documents/code/fhmm/v1")

In [2]:
from importlib import reload
from src.classifiers_davidjames9610.test_a.config import config_location
import importlib
import src.classifiers_davidjames9610.test_a.config as base_config
reload(base_config)
config = importlib.import_module(base_config.config_location)
reload(config)
print(config.basedir)
import time

/Users/david/Documents/code/fhmm/v1/src/classifiers_davidjames9610/test_a/speech_commands


In [3]:
import pickle
import d_classifiers_to_results
reload(d_classifiers_to_results)

sls = pickle.load(open(config.samples_labels,'rb'))
features = pickle.load(open(config.features,'rb'))
classifiers = pickle.load(open(config.classifiers,'rb'))
results = pickle.load(open(config.results,'rb'))


In [4]:
import src.misc_davidjames9610.utils as utils
reload(utils)

# plot grouped bar graph

stat_type = 'ACC'
features_to_include = ['lp', 'lp_GaussNoise_SNR20', 'lp_GaussNoise_SNR40']
y_axis_bins = {}

for result_key in results:
    curr_results = results[result_key]
    for classifier_key in curr_results:
        curr_classifier = curr_results[classifier_key]
        for feature in curr_classifier:
            curr_cm = curr_classifier[feature]
            curr_stats = curr_classifier[feature]['stats']
            print('')
            # acc_measure = utils.get_performance_metrics()

KeyboardInterrupt: 